In [4]:
# 1. 구글 드라이브 마운트
from google.colab import drive
drive.mount('/content/drive')

# 2. Ultralytics 라이브러리 설치
!pip install ultralytics

Mounted at /content/drive
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 30.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 104.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 90.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 49.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.7/188.7 MB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 59.2 MB/s eta 0:00:00
  Attempting uninsta

In [1]:
import os
import shutil
import glob

# --------------------------------------------------------------------------
# 1. 설정: 다운로드할 데이터셋 정보와 새로운 클래스 ID를 정의합니다.
# Roboflow에서 복사한 코드에서 workspace, project, version, api_key 값을 가져와 채워주세요.
# --------------------------------------------------------------------------
datasets_to_download = {
    "spider":    {"workspace": "marks-workspace-ciafp", "project": "spider-oojhf-y7bpu", "version": 1, "class_id": 3},
    "ant":       {"workspace": "marks-workspace-ciafp", "project": "ant-detection-rgchz-nq734", "version": 1, "class_id": 2},
    "cockroach": {"workspace": "marks-workspace-ciafp", "project": "cockroach-u9qmu-hudbc", "version": 1, "class_id": 0},
    "fly":       {"workspace": "marks-workspace-ciafp", "project": "fly-u203t-g8hrz", "version": 1, "class_id": 1},
    "cockroach_hard": {"workspace": "ai-club-6te9f", "project": "cockroachdetector", "version": 2, "class_id": 0}
    # 여기에 다른 데이터셋을 추가할 수 있습니다.
}

# 최종 통합 데이터셋이 저장될 경로
final_dataset_dir = "/content/pest_dataset_final"

# --------------------------------------------------------------------------
# 2. 데이터셋 다운로드 및 통합 자동화
# --------------------------------------------------------------------------
!pip install -q roboflow

from roboflow import Roboflow

# 최종 데이터셋 폴더 구조 생성
for split in ['train', 'val']:
    os.makedirs(f"{final_dataset_dir}/images/{split}", exist_ok=True)
    os.makedirs(f"{final_dataset_dir}/labels/{split}", exist_ok=True)

# Roboflow API 키 입력 (Roboflow 계정 설정에서 확인 가능)
rf = Roboflow(api_key="hQ1UrcbW701sraVNtwjh") # <--- 여기에 실제 API 키를 넣으세요.

for name, info in datasets_to_download.items():
    print(f"--- Processing {name} dataset ---")

    # 데이터셋 다운로드
    project = rf.workspace(info["workspace"]).project(info["project"])
    dataset = project.version(info["version"]).download("yolov8")

    # 다운로드된 경로 (예: /content/Spider-Detection-1)
    source_dir = dataset.location
    new_class_id = info["class_id"]

    # train/val 분할에 대해 반복
    for split in ['train', 'val', 'test']:
        image_source_path = f"{source_dir}/{split}/images/"
        label_source_path = f"{source_dir}/{split}/labels/"

        if not os.path.exists(image_source_path):
            continue

        # 'test' 폴더의 데이터는 'train'으로 합쳐 데이터 양을 늘립니다.
        target_split = 'train' if split == 'test' else split

        # 레이블 파일 처리: 클래스 ID 변경
        for label_file in glob.glob(f"{label_source_path}/*.txt"):
            with open(label_file, 'r') as f:
                lines = f.readlines()

            with open(label_file, 'w') as f:
                for line in lines:
                    # 각 줄의 맨 앞 숫자(기존 클래스 ID)를 새로운 ID로 변경
                    parts = line.strip().split()
                    parts[0] = str(new_class_id)
                    f.write(" ".join(parts) + "\n")

            # 수정한 파일을 최종 목적지로 이동
            shutil.move(label_file, f"{final_dataset_dir}/labels/{target_split}/")

        # 이미지 파일 이동
        for image_file in glob.glob(f"{image_source_path}/*.*"):
            shutil.move(image_file, f"{final_dataset_dir}/images/{target_split}/")

print("\n--- All datasets have been merged successfully! ---")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.6/88.6 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.8/66.8 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.9/49.9 MB 20.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 56.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 76.3 MB/s eta 0:00:00
--- Processing spider dataset ---
loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to Spider-1 in yolov8:: 100%|██████████| 406/406 [00:00<00:00, 3755.14it/s]

--- Processing ant dataset ---
loading Roboflow workspace...


loading Roboflow project...



Extracting Dataset Version Zip to Ant-Detection-1 in yolov8:: 100%|██████████| 364/364 [00:00<00:00, 9678.51it/s]


...


Extracting Dataset Version Zip to Fly-1 in yolov8:: 100%|██████████| 244/244 [00:00<00:00, 11886.85it/s]

--- Processing cockroach_hard dataset ---
loading Roboflow workspace...


loading Roboflow project...



Extracting Dataset Version Zip to cockroachDetector-2 in yolov8:: 100%|██████████| 898/898 [00:02<00:00, 309.81it/s]



--- All datasets have been merged successfully! ---


In [5]:
import os
import glob
import shutil
import random

# 1. 경로 설정
source_train_images_dir = "/content/pest_dataset_final/images/train/"
source_train_labels_dir = "/content/pest_dataset_final/labels/train/"

dest_val_images_dir = "/content/pest_dataset_final/images/val/"
dest_val_labels_dir = "/content/pest_dataset_final/labels/val/"

# 2. 훈련 데이터의 20%를 검증 데이터로 설정
split_ratio = 0.2

# 3. 훈련 이미지 파일 목록 가져오기
all_train_images = glob.glob(f"{source_train_images_dir}/*.*")
random.shuffle(all_train_images) # 파일을 무작위로 섞음

# 4. 이동할 파일 개수 계산
num_to_move = int(len(all_train_images) * split_ratio)
files_to_move = all_train_images[:num_to_move]

print(f"Total train images: {len(all_train_images)}")
print(f"Moving {num_to_move} images to validation set...")

# 5. 선택된 파일들을 val 폴더로 이동
for img_path in files_to_move:
    # 이미지 파일 이동
    shutil.move(img_path, dest_val_images_dir)

    # 해당 이미지에 맞는 레이블(.txt) 파일 경로 찾기
    base_filename = os.path.basename(img_path)
    label_filename = os.path.splitext(base_filename)[0] + ".txt"
    label_path = os.path.join(source_train_labels_dir, label_filename)

    # 레이블 파일도 함께 이동
    if os.path.exists(label_path):
        shutil.move(label_path, dest_val_labels_dir)

print("\n--- Successfully created validation set! ---")
print(f"New train image count: {len(glob.glob(f'{source_train_images_dir}/*.*'))}")
print(f"New val image count: {len(glob.glob(f'{dest_val_images_dir}/*.*'))}")

Total train images: 736
Moving 147 images to validation set...

--- Successfully created validation set! ---
New train image count: 589
New val image count: 330


In [ ]:
# data 인자에 새로운 yaml 파일 경로를 지정합니다.
# name으로 이번 훈련의 결과가 저장될 폴더 이름을 정해줍니다.
# !yolo task=detect mode=train model=yolov8m.pt data=/content/drive/MyDrive/Pest-Detection-YOLOv8/pest_config_final.yaml epochs=150 imgsz=640 name=pest_final_model project=/content/drive/MyDrive/Pest-Detection-YOLOv8/runs

Ultralytics 8.3.177 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/drive/MyDrive/Pest-Detection-YOLOv8/pest_config_final.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=150, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8m.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=pest_final_model3, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=100

In [ ]:
# resume 옵션. 다시 훈련 시작
# 'resume' 옵션을 사용하여 마지막으로 저장된 모델('last.pt')을 불러옵니다.
# !yolo task=detect mode=train resume model=/content/drive/MyDrive/Pest-Detection-YOLOv8/runs/pest_final_model/weights/last.pt

In [ ]:
# 1. 테스트할 이미지를 구글 드라이브에 업로드합니다. (예: new_pest_image.jpg)
# 2. 아래 코드를 새 셀에서 실행합니다.

# !yolo task=detect mode=predict model=/content/drive/MyDrive/Pest-Detection-YOLOv8/runs/pest_final_model3/weights/best.pt source=/content/drive/MyDrive/Pest-Detection-YOLOv8/ant.jpg project='/content/drive/MyDrive/Pest-Detection-YOLOv8' name='prediction_results'

Ultralytics 8.3.177 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
Model summary (fused): 92 layers, 25,842,076 parameters, 0 gradients, 78.7 GFLOPs

image 1/1 /content/drive/MyDrive/Pest-Detection-YOLOv8/ant.jpg: 448x640 (no detections), 53.9ms
Speed: 3.6ms preprocess, 53.9ms inference, 21.8ms postprocess per image at shape (1, 3, 448, 640)
Results saved to /content/drive/MyDrive/Pest-Detection-YOLOv8/prediction_results
💡 Learn more at https://docs.ultralytics.com/modes/predict


## 문제 1: 거대 거미의 출현 (심각한 오탐지)
현상: 모델이 여러 마리의 바퀴벌레가 뭉쳐있는 영역 전체를 하나의 '거미'라고, 그것도 꽤 높은 확신도(0.72)로 예측했습니다.

원인: 모델은 훈련 과정에서 한 마리씩 떨어져 있는 해충 이미지만을 보았을 가능성이 높습니다. 수십 마리의 바퀴벌레가 서로 겹치고 얽혀있는 복잡한 패턴은 모델이 한 번도 본 적 없는 새로운 데이터입니다. 따라서 모델은 이 복잡하게 얽힌 다리와 몸통의 패턴을 보고, 자기가 아는 것 중에서 가장 특징이 비슷한 **'다리가 많은 거미'**라고 잘못된 결론을 내린 것입니다.

## 문제 2: 소심한 바퀴벌레 탐지 (낮은 확신도와 대량 누락)
현상: 모델이 올바르게 '바퀴벌레'라고 인식한 개체들은 확신도가 0.31로 매우 낮으며, 이미지에 있는 수십 마리 중 단 두 마리만 찾아냈습니다.

원인:

낮은 재현율(Recall)의 실현: 이것은 우리가 이전에 최종 성적표에서 확인했던 바퀴벌레 클래스의 낮은 재현율(Recall: 0.786) 문제가 실제로 나타난 것입니다. 모델이 바퀴벌레를 보고도 확신이 없어 그냥 지나치는 경우가 많다는 의미입니다.

겹침 및 가려짐(Occlusion): 대부분의 바퀴벌레가 서로의 몸에 의해 일부가 가려져 있습니다. 모델이 주로 온전한 형태의 해충 이미지만 학습했다면, 이렇게 일부만 보이는 객체는 인식하기 어려워합니다.

## 해결책: 모델에게 '어려운 문제' 가르치기
이 두 가지 문제를 한 번에 해결할 수 있는 가장 확실한 방법은, 모델이 어려워하는 상황을 학습 데이터에 추가하여 다시 훈련시키는 것입니다.

'어려운' 데이터 보강:

여러 마리가 겹쳐 있거나 모여 있는 이미지를 집중적으로 수집하여 데이터셋에 추가합니다. (가장 중요!)

해충의 일부만 보이거나 가려진 이미지를 추가합니다.

다양한 각도와 어두운 조명에서 찍힌 이미지를 추가합니다.

모델 재훈련:

이렇게 '어려운 문제'들이 포함된 데이터셋으로 모델을 다시 훈련시키면, 모델은 더 이상 해충 무리를 보고 당황하여 '거미'라고 부르지 않을 것입니다. 또한, 겹쳐 있는 개체들도 더 자신감 있게 '바퀴벌레'라고 인식하게 될 것입니다.

이 이미지는 "우리 모델을 더 똑똑하게 만들려면 어떤 데이터가 더 필요한가?"라는 질문에 대한 매우 명확한 정답을 알려주고 있습니다.


그런 어려운 데이터는 어디서 어떻게 찾아? 얼마나 찾아?




생각하는 과정 표시
네, 아주 현실적이고 중요한 질문입니다. '좋은 데이터'를 구하는 것이 AI 모델 개발의 성패를 가른다고 해도 과언이 아니니까요.

'어려운 데이터'를 효율적으로 찾고, 얼마나 추가해야 할지에 대한 실질적인 가이드를 알려드리겠습니다.

## 1. '어려운 데이터' 찾는 방법
### 방법 1: 검색 키워드 바꾸기 (가장 기본)
단순히 'cockroach' 라고만 검색하면 깨끗하고 선명한 한 마리짜리 이미지만 나옵니다. 실제 현장의 지저분한 상황을 묘사하는 키워드로 바꿔야 합니다.

효과적인 검색 키워드 (영어로 검색 시 훨씬 많은 결과):

cockroach infestation (바퀴벌레 대량 발생)

pest glue trap (해충 끈끈이 트랩)

cockroach cluster (바퀴벌레 무리)

ants trail (개미 행렬)

spider nest / spider web (거미집)

구글 이미지 검색에서 이런 키워드를 사용하면, 우리가 원하는 여러 마리가 겹쳐있거나 지저분한 배경의 이미지들을 찾을 확률이 훨씬 높아집니다.

### 방법 2: 동영상에서 황금 캐기 (가장 효과적)
**가장 추천하는 '비밀 무기'**입니다. 해충 방제(Pest Control) 영상에는 우리가 원하는 모든 '어려운 상황'이 담겨 있습니다.

YouTube 검색: 위에서 언급한 키워드(예: cockroach infestation)로 YouTube에 검색합니다.

영상 재생 및 캡처: 영상에서 바퀴벌레 무리가 움직이거나, 트랩에 여러 마리가 붙어있는 등 결정적인 장면이 나올 때마다 스크린샷을 찍습니다.

데이터 대량 확보: 단 1분의 영상에서도 수십 장의 서로 다른 각도와 겹침을 가진 고품질 훈련 이미지를 얻을 수 있습니다.

### 방법 3: 데이터 직접 합성하기 (고급)
기존에 가지고 있는 데이터를 활용하는 방법입니다.

'Copy-Paste' 증강: 깨끗한 이미지에서 바퀴벌레 한 마리를 오려내어, 다른 바퀴벌레 이미지에 여러 마리 붙여넣는 방식입니다. 이렇게 하면 '겹쳐있는' 상황을 인위적으로 만들 수 있습니다. Roboflow 같은 툴은 이러한 데이터 합성 기능을 제공하기도 합니다.

## 2. 얼마나 찾아야 할까?
정해진 답은 없지만, 목표를 정하고 효율적으로 접근하는 방법은 있습니다.

### 목표 설정: 가장 약한 부분부터 보강하기
가장 시급한 문제는 '바퀴벌레'의 낮은 재현율과 '무리' 인식 실패였습니다. 따라서 다른 해충보다는 바퀴벌레 데이터, 특히 '어려운' 바퀴벌레 데이터를 집중적으로 보강해야 합니다.

규칙 1: 기존 데이터의 20~30%를 목표로

가장 문제가 되었던 '바퀴벌레' 클래스를 기준으로, 현재 가지고 있는 바퀴벌레 훈련 이미지 수의 20~30% 정도를 추가하는 것을 첫 목표로 삼는 것이 좋습니다.

예를 들어, 기존 바퀴벌레 훈련 이미지가 200장이었다면, 새로운 '어려운' 이미지 40~60장을 추가하는 것을 목표로 설정합니다.

### 접근법: 점진적이고 반복적인 개선
한 번에 수천 장을 모으려고 하기보다는, 조금씩 추가하고, 다시 훈련하고, 성능을 평가하는 반복적인 접근이 훨씬 효율적입니다.

1차 목표 달성: '어려운' 바퀴벌레 이미지 50장을 추가합니다.

모델 재훈련: 추가된 데이터셋으로 모델을 다시 훈련시킵니다. (에포크는 100 정도로 줄여서 빠르게 테스트)

성능 평가: 이전과 같은 '실패 이미지'를 넣어보고 문제가 개선되었는지 확인합니다.

반복 또는 확장: 문제가 해결되었다면 다른 해충으로 확장하고, 여전히 부족하다면 '어려운' 이미지를 조금 더 추가하여 다시 훈련합니다.

물론, 새로 찾은 모든 이미지에는 직접 바운딩 박스를 그리는 '레이블링' 작업을 해주어야 합니다. 이 과정을 통해 모델은 현실의 복잡하고 어려운 문제들을 해결하는 능력을 점차 갖추게 될 것입니다

## 모델이 개미를 찾지 못하는 이유
### 1. '상황'의 차이: 먹이 활동과 밀집
모델은 아마도 한 마리씩 기어가는 개미의 모습을 주로 학습했을 것입니다.

하지만 이 이미지는 수많은 개미가 먹이 주위에 떼로 뭉쳐 있고, 서로 겹쳐 있으며, 심지어 알로 보이는 하얀 물체까지 섞여 있습니다. 이런 복잡한 상황은 모델에게 매우 생소한 '돌발 문제'입니다.

### 2. '모습'의 차이: 색상과 배경
훈련 데이터에 있던 개미는 주로 검은색이었을 수 있지만, 사진 속 개미는 붉은빛을 띠는 다른 종일 수 있습니다.

또한, 끈끈이 트랩으로 보이는 선명한 노란색 배경 역시 모델이 학습하지 못한 생소한 환경입니다.

### 3. 낮은 확신도 임계값 (Confidence Threshold)
YOLO 모델은 기본적으로 확신도 점수가 0.25 이상인 것만 결과로 보여줍니다.

모델이 이미지 속 개미들을 보고 "개미 같긴 한데... 한 15% 정도만 확신할 수 있겠다 (ant 0.15)" 라고 판단했다면, 이 결과는 최종 출력에서 무시됩니다.

## 해결 방법: 2단계 접근법
### 1단계: 모델의 속마음 확인하기 (즉시 테스트)
먼저, 모델이 정말 아무것도 못 본 것인지, 아니면 확신이 없어서 말을 안 한 것인지 확인해봐야 합니다. 확신도 임계값(conf)을 낮춰서 다시 예측을 실행해 보세요.

아래 코드를 새 셀에 복사하여 실행합니다.

Python

# 확신도 기준을 0.1로 낮춰서 다시 예측을 실행합니다.
!yolo task=detect mode=predict \
      model='/content/drive/MyDrive/Pest-Detection-YOLOv8/runs/pest_final_model3/weights/best.pt' \
      source='/content/drive/MyDrive/Pest-Detection-YOLOv8/ant.jpg' \
      project='/content/drive/MyDrive/Pest-Detection-YOLOv8' \
      name='prediction_results_low_conf' \
      exist_ok=True \
      conf=0.1
conf=0.1: 확신도가 10%만 넘어도 일단 보여달라는 명령어입니다.

이렇게 하면 모델이 '긴가민가'했던 탐지 결과까지 모두 보여줍니다. 아마도 아주 낮은 점수(0.1~0.2)와 함께 개미 주변에 박스가 나타날 수 있습니다.

### 2단계: '오답 노트' 만들기 (근본적인 해결책)
이 문제를 근본적으로 해결하려면, 모델이 어려워하는 이 상황을 학습시켜야 합니다.

'어려운' 개미 데이터 추가:

다양한 종류/색상의 개미 이미지

개미들이 먹이에 떼로 모여있는 이미지 (가장 중요!)

개미집 주변이나 개미 행렬 이미지

레이블링 및 재훈련:

이 이미지처럼 탐지에 실패한 이미지들을 '오답 노트' 삼아 직접 레이블링하여 훈련 데이터셋에 추가합니다.

보강된 데이터셋으로 모델을 다시 훈련시키면, 다음부터는 이런 상황에서도 자신 있게 개미를 찾아낼 것입니다.

이러한 '실패' 사례들은 우리 모델의 약점을 정확히 알려주는 보물과도 같습니다. 이 피드백을 바탕으로 모델을 개선해나가면 됩니다.

In [6]:
# model 인자에 이전 훈련의 결과물 경로를 지정하여 '전이 학습'을 수행합니다.
# name을 새로 지정하여 이전 결과와 겹치지 않게 관리합니다.

!yolo task=detect mode=train \
      model='/content/drive/MyDrive/Pest-Detection-YOLOv8/runs/pest_final_model3/weights/best.pt' \
      data='/content/drive/MyDrive/Pest-Detection-YOLOv8/pest_config_final.yaml' \
      epochs=100 \
      imgsz=640 \
      project='/content/drive/MyDrive/Pest-Detection-YOLOv8/runs' \
      name='pest_model_v2_cockroach_fix' \
      exist_ok=True

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
Ultralytics 8.3.178 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/drive/MyDrive/Pest-Detection-YOLOv8/pest_config_final.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=100, erasing=0.4, exist_ok=True, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8